In [22]:
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')
from collections import defaultdict

In [23]:
import sys
sys.path.append('code/')

In [24]:
from code_bj import *

In [28]:
class Dealer():
    def __init__( self, verbose = True):
        self.cards = np.array( [])
        self.value = 0
        self.verbose = verbose
    
    def draw( self):
        card = draw_card()
        self.cards = np.append( self.cards, card)
        self.possible_values = value_sum( self.cards)
        if min( self.possible_values) <= 21:
            self.value = max( self.possible_values[ self.possible_values <= 21])
        else:
            self.value = min( self.possible_values)
        if self.verbose == True:
            print( f'Dealer\'s cards are {self.cards}')

In [32]:
class Player():
    def __init__( self, verbose = True):
        self.cards = np.array( [])
        self.value = 0
        self.verbose = verbose
    
    def draw( self):
        card = draw_card()
        self.cards = np.append( self.cards, card)
        self.possible_values = value_sum( self.cards)
        if min( self.possible_values) <= 21:
            self.value = max( self.possible_values[ self.possible_values <= 21])
        else:
            self.value = min( self.possible_values)
        if verbose == True:
            print( f'Player\'s cards are {self.cards}')

In [33]:
def draw_card():
    global current_deck
    global running_count
    card = np.random.choice( current_deck)
    current_deck.remove( card)
    running_count += HiLo( card)
    return card

In [116]:
def count():
    global player
    global dealer
    global running_count
    global true_count
    global betsize
    
    true_count = running_count // ( len( current_deck) / 52)
    
    if true_count <= 1:
        betsize = 1
    if true_count == 2:
        betsize = 2
    if true_count == 3:
        betsize = 3
    if true_count == 4:
        betsize = 4
    if true_count >= 5:
        betsize = 5
    
    return betsize

In [117]:
def game( player_upcard = None, dealer_upcard = None, betsize = 1):
    global current_deck
    global player
    global dealer
    global running_count
    global true_count
    
    surrender = False
    
    change = 0
    dealer = Dealer( verbose = False)
    player = Player( verbose = False)
    
    if dealer_upcard == None:
        dealer.draw()
    
    if player_upcard == None:
        player.draw()
    else:
        np.append( player.cards, player_upcard) # In case of splitting
        
    player.draw()

    action = 'H'

    while (player.value < 21) and (action != 'S'):
        
        if (len( player.cards) == 2) and min( player.cards) == max( player.cards):
            total = 'pair'
        else:
            if 'A' in player.cards:
                total = 'soft'
            else:
                total = 'hard'

        action = basic_strategy_action( player, dealer, total)

        if action == 'Sur':
            if len( player.cards) == 2:
                surrender = True
                change = -betsize / 2
                return change
            else:
                action = 'H'
        
        if action == 'X':
            splitted_card = min( player.cards)
            # if splitted_card == 'A':
                
            return game( player_upcard = min( player.cards))
        
        if action == 'H':
            player.draw()

        if action == 'D':
            if len( player.cards) == 2:
                player.draw()
                betsize *= 2
                action = 'S'
            else:
                player.draw()

    while dealer.value < 17:
        dealer.draw()
    
    if (len( player.cards) == 2) and (set( player.cards) == { 'A', '10'}): # Player getting a Blackjack
        change = betsize * (3/2)
    elif ( (player.value < 22) & (player.value > dealer.value) ) | (dealer.value >= 22):
        change = betsize
    elif (player.value >= 22) | (player.value < dealer.value):
        change = -betsize
    
    if len( current_deck) < 40:
        current_deck = (deck * 2).copy()
        running_count = 0
        # print( 'shuffle')
        
    return change

In [118]:
verbose = False

record = defaultdict( lambda: record_run.copy())

for i in range( 100):
    result = np.array( [])
    bankroll = 30
    current_deck = (deck * 2).copy()
    running_count = 0
    true_count = 0
    
    for j in range( 300):
        
        if bankroll > 0:
            change = game()
        else:
            change = 0

        bankroll = bankroll + change
        record_run[ j][ 'run_num'] = j
        record_run[ j][ 'player_cards'] = player.cards
        record_run[ j][ 'dealer_cards'] = dealer.cards
        record_run[ j][ 'betsize'] = betsize
        record_run[ j][ 'change'] = change
        record_run[ j][ 'running_count'] = running_count
        record_run[ j][ 'true_count'] = true_count
        
        result = np.append( result, bankroll)

        #print( f'Bankroll is {bankroll}.')
        #print()
    record[i] = result

In [119]:
record_melted = pd.DataFrame( record).reset_index().melt( id_vars = 'index', value_vars = list( range( 100)))
record_melted

,index,variable,value
0,0,0,29.0
1,1,0,29.0
2,2,0,28.5
3,3,0,29.5
4,4,0,31.5
...,...,...,...
29995,295,99,14.5
29996,296,99,13.5
29997,297,99,13.5
29998,298,99,12.5


In [120]:
alt.Chart( record_melted.query( 'index<1000')).mark_line().encode(
    x = 'index',
    y = 'value',
    color = 'variable:N')

alt.Chart(...)

In [121]:
pd.DataFrame( record).iloc[ 299].sort_values().head(30).reset_index()

,index,299
0,3,-0.5
1,53,0.0
2,32,4.5
3,99,12.5
4,56,17.0
5,25,20.0
6,38,23.0
7,62,23.5
8,50,25.0
9,17,28.5


In [122]:
record_mean = pd.DataFrame( record).mean( axis = 1).reset_index()
record_mean.columns = [ 'index', 'bankroll']
alt.Chart( record_mean).mark_line().encode(
    x = 'index',
    y = 'bankroll')

alt.Chart(...)

# Storing

In [123]:
def game( player_upcard = None, dealer_upcard = None, betsize = 1):
    global current_deck
    global player
    global dealer
    global running_count
    global true_count
    
    surrender = False
    
    change = 0
    dealer = Dealer( verbose = False)
    player = Player( verbose = False)
    
    if dealer_upcard == None:
        dealer.draw()
    
    if player_upcard == None:
        player.draw()
    else:
        np.append( player.cards, player_upcard) # In case of splitting
        
    player.draw()

    action = 'H'

    while (player.value < 21) and (action != 'S'):
        
        if (len( player.cards) == 2) and min( player.cards) == max( player.cards):
            total = 'pair'
        else:
            if 'A' in player.cards:
                total = 'soft'
            else:
                total = 'hard'

        action = basic_strategy_action( player, dealer, total)

        if action == 'Sur':
            if len( player.cards) == 2:
                surrender = True
                change = -betsize / 2
                return change
            else:
                action = 'H'
        
        if action == 'X':
            splitted_card = min( player.cards)
            # if splitted_card == 'A':
                
            return game( player_upcard = min( player.cards))
        
        if action == 'H':
            player.draw()

        if action == 'D':
            if len( player.cards) == 2:
                player.draw()
                betsize *= 2
                action = 'S'
            else:
                player.draw()

    while dealer.value < 17:
        dealer.draw()
    
    if (len( player.cards) == 2) and (set( player.cards) == { 'A', '10'}): # Player getting a Blackjack
        change = betsize * (3/2)
    elif ( (player.value < 22) & (player.value > dealer.value) ) | (dealer.value >= 22):
        change = betsize
    elif (player.value >= 22) | (player.value < dealer.value):
        change = -betsize
    
    if len( current_deck) < 40:
        current_deck = (deck * 2).copy()
        running_count = 0
        # print( 'shuffle')
        
    return change

In [124]:
record_run = defaultdict( lambda: record_game.copy())

verbose = False

record = dict()

bankroll = 20
current_deck = (deck * 2).copy()
betsize = 1

running_count = 0
true_count = 0

for j in range( 200):
    
    betsize = count()
    
    record_run[ j][ 'running_count'] = running_count
    record_run[ j][ 'remaining_deck'] = ( len( current_deck) / 52)
    record_run[ j][ 'true_count'] = true_count

    change = game( betsize = betsize)

    bankroll = bankroll + change
    record_run[ j][ 'run_num'] = j
    record_run[ j][ 'player_cards'] = player.cards
    record_run[ j][ 'dealer_cards'] = dealer.cards
    record_run[ j][ 'betsize'] = betsize
    record_run[ j][ 'change'] = change

    result = np.append( result, bankroll)

In [125]:
pd.DataFrame( record_run).T

,run_num,player_cards,dealer_cards,change,running_count,true_count,betsize,remaining_deck
0,0,"[5, 9]","[3, 6, 10]",-1,0,0.0,1,2.0
1,1,"[10, 8]","[7, 10]",1,2,1.0,1,1.903846
2,2,"[3, 10, 3, 6]","[10, 10]",-1,0,0.0,1,1.826923
3,3,"[8, 10]","[3, A, 3]",1,0,0.0,1,1.711538
4,4,"[7, 2, 3]","[5, 10, 2]",-2,0,0.0,1,1.615385
...,...,...,...,...,...,...,...,...
195,195,"[5, 7, 10]","[10, 10]",-1,1,1.0,1,0.903846
196,196,"[6, 10]","[5, 9, 9]",1,-1,-2.0,1,0.807692
197,197,"[8, 5, 2, A, 7]","[A, 10]",-1,0,0.0,1,2.0
198,198,"[A, 8]","[10, A]",-1,-1,-1.0,1,1.865385


In [127]:
pd.DataFrame( record_run).T.to_csv( '11-08-0944.csv')

# Old codes

In [128]:
verbose = False

length = []
for i in range( 100):
    current_deck = (deck * 2).copy()
    bankroll = 20
    result = np.array( [])
    while bankroll > 0:
        surrender = False
        betsize = 1
        dealer = Dealer( verbose = False)
        player = Player( verbose = False)

        dealer.draw()
        player.draw()
        player.draw()

        action = 'H'

        while (player.value < 21) and (action != 'S'):
            if 'A' in player.cards:
                total = 'soft'
            else:
                total = 'hard'
                
            action = basic_strategy_action( player, dealer, total)
            
            if action == 'Sur':
                if len( player.cards) == 2:
                    surrender = True
                    break
                else:
                    action = 'H'

            if action == 'H':
                player.draw()

            if action == 'D':
                player.draw()
                betsize *= 2
                action = 'S'

        if surrender == True:
            bankroll -= betsize / 2
            continue

        while dealer.value < 17:
            dealer.draw()

        if len( current_deck) < 50:
            current_deck = (deck * 2).copy()

        if ( (player.value < 22) & (player.value > dealer.value) ) | (dealer.value >= 22):
            bankroll += betsize
        elif (player.value >= 22) | (player.value < dealer.value):
            bankroll -= betsize

        result = np.append( result, bankroll)

        #print( f'Bankroll is {bankroll}.')
        #print()
    length.append( len( result))

In [79]:
if len( current_deck) < 50:
    current_deck = (deck * 2).copy()

In [ ]:
current_deck

In [ ]:
values = np.array( [], dtype = int)
num_A = np.count_nonzero( player.cards == 'A')
values = np.append( values, sum( list( map( value, player.cards))))
for i in range( 1, num_A+1):
    values = np.append( values, sum( list( map( value, player.cards)))-10*i)
values[ values <= 21]

In [ ]:
player.value, dealer.value

In [ ]:
pd.Series( length).describe()

In [ ]:
pd.Series( length).hist( bins = 100)